In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau

from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras.applications import MobileNetV2,EfficientNetB0
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory('../input/face-mask-detection/Dataset', validation_split=0.2, subset="training", shuffle=True, seed = 2021, labels='inferred', batch_size=32, image_size=(224,224))
val_ds = tf.keras.preprocessing.image_dataset_from_directory('../input/face-mask-detection/Dataset', validation_split=0.2, subset="validation", shuffle=True, seed = 2021, labels='inferred', batch_size=32, image_size=(224,224))

Found 8982 files belonging to 3 classes.
Using 7186 files for training.


2021-10-22 12:27:55.454805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 12:27:55.553134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 12:27:55.554152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 12:27:55.564582: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 8982 files belonging to 3 classes.
Using 1796 files for validation.


In [3]:
train_ds.class_names

['mask_weared_incorrect', 'with_mask', 'without_mask']

In [4]:
class_name = ['Incorrect','With_Mask','Without_Mask']

In [5]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

2021-10-22 12:27:59.780817: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(32, 224, 224, 3)
(32,)


In [6]:
image_batch, labels_batch = next(iter(train_ds))
image_batch[0], labels_batch[0]

(<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
 array([[[  5.        ,   3.        ,   4.        ],
         [ 18.214289  ,  16.214289  ,  17.214289  ],
         [ 39.357143  ,  37.357143  ,  38.357143  ],
         ...,
         [125.14287   , 125.14287   , 125.14287   ],
         [126.28572   , 126.28572   , 126.28572   ],
         [127.        , 127.        , 127.        ]],
 
        [[  3.5714283 ,   1.9285712 ,   2.5714283 ],
         [ 13.469389  ,  11.826532  ,  12.469389  ],
         [ 29.306122  ,  27.663265  ,  28.306122  ],
         ...,
         [126.57144   , 126.57144   , 126.57144   ],
         [127.714294  , 127.714294  , 127.714294  ],
         [128.42857   , 128.42857   , 128.42857   ]],
 
        [[  1.2857141 ,   0.21428561,   0.28571415],
         [  5.877551  ,   4.806123  ,   4.877551  ],
         [ 13.224489  ,  12.153061  ,  12.224489  ],
         ...,
         [128.85715   , 128.85715   , 128.85715   ],
         [130.        , 130.        , 130.      

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = tf.keras.Sequential()

model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))
model.add(MobileNetV2(weights="imagenet", include_top=False))

model.add(layers.AveragePooling2D(pool_size=(7, 7)))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(3, activation='softmax'))

9420800/9406464 [==============================] - 0s 0us/step


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, None, None, 1280)  2257984   
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
save_best = tf.keras.callbacks.ModelCheckpoint("MaskDetection.h5",monitor='val_accuracy',save_best_only=True, verbose=1)

model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[save_best])

Epoch 1/10


2021-10-22 12:28:08.440904: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


225/225 [==============================] - 54s 193ms/step - loss: 0.1551 - accuracy: 0.9507 - val_loss: 0.8506 - val_accuracy: 0.9254

Epoch 00001: val_accuracy improved from -inf to 0.92539, saving model to MaskDetection.h5
Epoch 2/10
225/225 [==============================] - 36s 161ms/step - loss: 0.0790 - accuracy: 0.9773 - val_loss: 0.7779 - val_accuracy: 0.9271

Epoch 00002: val_accuracy improved from 0.92539 to 0.92706, saving model to MaskDetection.h5
Epoch 3/10
225/225 [==============================] - 36s 159ms/step - loss: 0.0563 - accuracy: 0.9825 - val_loss: 11.2535 - val_accuracy: 0.5473

Epoch 00003: val_accuracy did not improve from 0.92706
Epoch 4/10
225/225 [==============================] - 36s 159ms/step - loss: 0.0355 - accuracy: 0.9875 - val_loss: 3.6872 - val_accuracy: 0.8224

Epoch 00004: val_accuracy did not improve from 0.92706
Epoch 5/10
225/225 [==============================] - 37s 165ms/step - loss: 0.0362 - accuracy: 0.9903 - val_loss: 3.4311 - val_accur

In [12]:
model.evaluate(val_ds)

57/57 [==============================] - 2s 28ms/step - loss: 0.3973 - accuracy: 0.9516


[0.3972527086734772, 0.9515590071678162]

In [13]:
for image_batch, labels_batch in val_ds:
    pred = model.predict(image_batch[8].numpy().reshape(-1,224,224,3))
    print(pred.round())
    print(labels_batch[8])
    break

[[0. 1. 0.]]
tf.Tensor(1, shape=(), dtype=int32)
